## Домашнее задание №4

Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec, Fastext, Adagram.Для обучения этих моделей можете воспользоваться корпусом новостных текстов, с которым мы работали на 4 и 5 семинарах. А можете использовать любой другой корпус (сами тексты соревнования использовать не надо).

У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары). 

Постройте обучающую выборку из этих близостей. Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру).   

Предложите любой способ улучшить эту метрику (и реализуйте его!).



In [ ]:
import pandas as pd
from lxml import html, etree
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import sys

In [ ]:
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)

In [ ]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [ ]:
def get_embedding_adagram(text, model, window, dim):
    text = text.split()
    
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v # просто умножаем вектор на частоту
        
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

### Корпус парафраз

In [ ]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [ ]:
data.head()

In [ ]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

### Корпус новостных текстов для обучения моделей

In [ ]:
import zipfile

with zipfile.ZipFile('news_texts.csv.zip') as z:
    with z.open("news_texts.csv") as f:
        train = pd.read_csv(f, header=0)

In [ ]:
train.dropna(inplace=True)

In [ ]:
train.head()

### Обучаем модели на новостных текстах (а для Adagram загружаем готовую модель)

In [310]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000,
                     #ngram_range = (1,2)
                    )
X = cv.fit_transform(train['content_norm'])

In [311]:
#cv = CountVectorizer(max_features=1000,
                    #min_df=5,
                    #max_df=0.8,
                    #lowercase=False,
                    #ngram_range=(1, 2))
#X = cv.fit_transform(train['content_norm'])

In [312]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000,
                        #ngram_range = (1,2)
                       )
X_tfidf = tfidf.fit_transform(train['content_norm'])

In [ ]:
#tdidf = TfidfVectorizer(max_features=1000,
                    #min_df=5,
                    #max_df=0.8,
                    #lowercase=False,
                    #ngram_range=(1, 2))
#X = cv.fit_transform(train['content_norm'])

In [313]:
svd = TruncatedSVD(50)
svd.fit(X)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)

In [314]:
nmf = NMF(50)
nmf.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=50, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [ ]:
fast_text = gensim.models.FastText([text.split() for text in train['content_norm']], size=50, min_n=4, max_n=8)

In [ ]:
w2v = gensim.models.Word2Vec([text.split() for text in train['content_norm']], size=50, sg=1)

In [ ]:
import adagram

In [ ]:
vm = adagram.VectorModel.load('out.pkl')

### Преобразуем тексты в векторы

In [315]:
X_text_1 = svd.transform(tfidf.transform(data['text_1_norm']))
X_text_2 = svd.transform(tfidf.transform(data['text_2_norm']))

In [316]:
X_text_1_nmf = nmf.transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.transform(tfidf.transform(data['text_2_norm']))

In [317]:
X_text_1_nmf.shape

(7227, 50)

In [ ]:
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

In [ ]:
dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

In [ ]:
dim = 50
X_text_1_ag = np.zeros((len(data['text_1_norm']), dim))
X_text_2_ag = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_ag[i] = get_embedding_adagram(text, vm, 5, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_ag[i] = get_embedding_adagram(text, vm, 5, dim)

### Считаем косинусную близость, строим выборку

In [ ]:
from scipy.spatial import distance
import sys

In [ ]:
#функция для подсчета косинусной близости по строкам-векторам для отдельных текстов в полученных ранее матрицах
def get_cos_vector(matrix_A, matrix_B):

    cos_array = np.zeros((len(matrix_A),1)) #len(matrix_A) == len(matrix_B)!!!

    for i in range(len(matrix_A)):
        cos_array[i] = cos_array[i] + distance.cosine(matrix_A[i], matrix_B[i])
        if cos_array[i] == 'nan':
            cos_array[i] = 0
        else:
            continue
    return np.nan_to_num(cos_array)

In [318]:
cos_SVD = get_cos_vector(X_text_1, X_text_2)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/opt/conda/lib/python3.6/site-packages/scipy/spatial/distance.py:649: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [319]:
cos_SVD.shape

(7227, 1)

In [320]:
cos_NMF = get_cos_vector(X_text_1_nmf, X_text_2_nmf)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/opt/conda/lib/python3.6/site-packages/scipy/spatial/distance.py:649: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [ ]:
cos_w2v = get_cos_vector(X_text_1_w2v, X_text_2_w2v)

In [ ]:
cos_FT = get_cos_vector(X_text_1_ft, X_text_2_ft)

In [ ]:
cos_AG = get_cos_vector(X_text_1_ag, X_text_2_ag)

In [321]:
cos = np.concatenate([cos_SVD, cos_NMF, cos_w2v, cos_FT, cos_AG], axis = 1)

In [322]:
cos.shape

(7227, 5)

### Обучаем модель (ЛогРег) на полученной выборке

In [323]:
y = data['label'].values

In [324]:
train_X, valid_X, train_y, valid_y = train_test_split(cos, y, random_state=1)

In [325]:
clf = LogisticRegression(C=1000, class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

             precision    recall  f1-score   support

         -1       0.64      0.61      0.62       629
          0       0.50      0.53      0.51       737
          1       0.48      0.46      0.47       441

avg / total       0.54      0.54      0.54      1807



In [326]:
print(np.mean(cross_val_score(clf, train_X, train_y, scoring="f1_micro")))

0.559594746896


### Пробуем улучшить качество обучения

In [327]:
clf_1 = LogisticRegression(C=2000,
                           class_weight='balanced',
                           multi_class='multinominal',
                           solver = 'newton-cg',
                           max_iter = 200,
                           n_jobs = 2,
                           warm_start = True
                          )

In [328]:
clf_1.fit(train_X, train_y)
preds_1 = clf_1.predict(valid_X)
print(classification_report(valid_y, preds_1))

             precision    recall  f1-score   support

         -1       0.65      0.59      0.62       629
          0       0.48      0.79      0.60       737
          1       1.00      0.04      0.07       441

avg / total       0.67      0.54      0.47      1807



In [329]:
print(np.mean(cross_val_score(clf_1, train_X, train_y, scoring="f1_micro")))

0.560882650894


### Результаты

Настройка параметров классификатора позволяет немного повысить микросреднюю f1-меру.
Также я попробовала добавить биграмы в tfidf векторайзер, а также немного "поиграться" с параметрами, но при этом результаты не улучшились